Evaluations with zero-shot, 5-shot, full fine-tune, merged LoRA, merged DoRA.

In [1]:
from pathlib import Path
import os, json
from safetensors.torch import save_file
import copy
from tqdm import tqdm
import safetensors
import safetensors.torch
from glob import glob
from transformers import AutoConfig, AutoTokenizer
from transformers.utils import hub, SAFE_WEIGHTS_NAME, SAFE_WEIGHTS_INDEX_NAME
from hqq.core.quantize import HQQLinear, HQQBackend, BaseQuantizeConfig, Quantizer
from fastcore.script import *

import bitsandbytes as bnb
from bitsandbytes.nn.modules import Params4bit
import torch

### Evaluate

In [2]:
import re
from datasets import load_dataset
from fastcore.parallel import parallel
from vllm import LLM, SamplingParams

In [3]:
def extract_last_number_or_ratio(s):
    # Find all sequences of digits, possibly with leading currency symbols, decimal points, and ratios
    patterns = re.findall(r'[\$€£]?\d+(?:\.\d+)?(?:\:\d+(?:\.\d+)?)?', s)
    
    # Return the last pattern found, or None if there are no matches
    if patterns:
        return patterns[-1]
    else:
        return None

In [4]:
def exact_match_score(preds, labels):
    return sum(p==g for p,g in zip(preds, labels))/len(preds)

In [5]:
dataset = load_dataset("microsoft/orca-math-word-problems-200k")['train'].shuffle(seed=42)
dataset = dataset.select(range(len(dataset)-5000,len(dataset)))
short_answers_gt = parallel(extract_last_number_or_ratio, dataset['answer'], progress=True)

In [6]:
valid_dataset = dataset.select(range(500))

In [7]:
inputs = [f"###Question:\n{question}\n###Answer:\n" for question in valid_dataset['question']]

In [8]:
labels = short_answers_gt[:500]

In [9]:
len(inputs), len(labels)

(500, 500)

In [10]:
NUM_GPUS = torch.cuda.device_count(); NUM_GPUS

8

In [11]:
TOKENIZER = "meta-llama/Meta-Llama-3-8B"
TOKENIZER = "meta-llama/Meta-Llama-3-8B-Instruct"

In [12]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
def convert_to_chat_input(question):
    messages = [
        {"role": "system", "content": "You are an AI assistant that excels in solving math problems."},
        {"role": "user", "content": question},
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [14]:
chat_inputs = [convert_to_chat_input(question) for question in valid_dataset['question']]

In [15]:
len(chat_inputs)

500

#### FINETUNED 

In [18]:
model_dir = "/workspace/models/"

In [19]:
# MODEL_NAME = "llama-3-8b-orca-math-10k-full" # 0.4
# MODEL_NAME = "llama-3-8b-orca-math-10k-bnb-qlora-merged" # 0.276
# MODEL_NAME = "llama-3-8b-orca-math-10k-bnb-qdora-merged" # 0.458

# MODEL_NAME = "llama-3-8b-orca-math-100k-bnb-qlora-merged"
# MODEL_NAME = "llama-3-8b-orca-math-100k-bnb-qdora-merged" # 0.558

# MODEL_NAME = "llama-3-8b-instruct-orca-math-10k-bnb-qlora-merged"
# MODEL_NAME = "llama-3-8b-instruct-orca-math-10k-bnb-qdora-merged"

# MODEL_NAME = "llama-3-8b-instruct-orca-math-10k-hqq-qdora-plus-merged"
# MODEL_NAME = "llama-3-8b-instruct-orca-math-10k-hqq-qdora-loftq-init-merged"
# MODEL_NAME = "llama-3-8b-instruct-orca-math-10k-hqq-qdora-plus-loftq-init-merged"

# MODEL_NAME = "/workspace/models/llama-3-8b-instruct-orca-math-10k-hqq-qlora-plus-merged"
# MODEL_NAME = "/workspace/models/llama-3-8b-instruct-orca-math-10k-hqq-qlora-loftq-init-merged"
# MODEL_NAME = "/workspace/models/llama-3-8b-instruct-orca-math-10k-hqq-qlora-plus-loftq-init-merged"

# MODEL_NAME = "/workspace/models/llama-3-8b-instruct-orca-math-10k-hqq-qdora-axis-1-merged"
MODEL_NAME = "/workspace/models/llama-3-8b-instruct-orca-math-10k-full"

In [21]:
llm = LLM(model=os.path.join(model_dir,MODEL_NAME), tokenizer=TOKENIZER, 
          tensor_parallel_size=NUM_GPUS, dtype="bfloat16")

2024-05-09 08:54:35,934	WARNING utils.py:580 -- Detecting docker specified CPUs. In previous versions of Ray, CPU detection in containers was incorrect. Please ensure that Ray has enough CPUs allocated. As a temporary workaround to revert to the prior behavior, set `RAY_USE_MULTIPROCESSING_CPU_COUNT=1` as an env var before starting Ray. Set the env var: `RAY_DISABLE_DOCKER_CPU_WARNING=1` to mute this warning.
2024-05-09 08:54:35,941	WARNING utils.py:592 -- Ray currently does not support initializing Ray with fractional cpus. Your num_cpus will be truncated from 64.6 to 64.
2024-05-09 08:54:36,071	INFO worker.py:1749 -- Started a local Ray instance.


INFO 05-09 08:54:37 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='/workspace/models/llama-3-8b-instruct-orca-math-10k-full', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=8, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=/workspace/models/llama-3-8b-instruct-orca-math-10k-full)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 05-09 08:55:18 utils.py:660] Found nccl from library /root/.config/vllm/nccl/cu12/libnccl.so.2.18.1
(RayWorkerWrapper pid=31465) INFO 05-09 08:55:18 utils.py:660] Found nccl from library /root/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 05-09 08:55:19 selector.py:81] Cannot use FlashAttention-2 backend because the flash_attn package is not found. Please install it for better performance.
INFO 05-09 08:55:19 selector.py:32] Using XFormers backend.
(RayWorkerWrapper pid=31465) INFO 05-09 08:55:21 selector.py:81] Cannot use FlashAttention-2 backend because the flash_attn package is not found. Please install it for better performance.
(RayWorkerWrapper pid=31465) INFO 05-09 08:55:21 selector.py:32] Using XFormers backend.
INFO 05-09 08:55:29 pynccl_utils.py:43] vLLM is using nccl==2.18.1
(RayWorkerWrapper pid=31465) INFO 05-09 08:55:29 pynccl_utils.py:43] vLLM is using nccl==2.18.1
(RayWorkerWrapper pid=31956) INFO 05-09 08:55:18 utils.py:660] Found nccl from library /root/.config/v

In [22]:
# base model
# outputs = llm.generate(inputs, SamplingParams(temperature=0.0, max_tokens=1024))

In [23]:
# chat model
outputs = llm.generate(chat_inputs, SamplingParams(temperature=0.0, max_tokens=1024, stop=["<|eot_id|>"]))

Processed prompts: 100%|██████████| 500/500 [01:26<00:00,  5.78it/s]


In [24]:
short_answers_pred = [extract_last_number_or_ratio(o.outputs[0].text) for o in outputs]

In [25]:
exact_match_score(short_answers_pred, labels)

0.528

#### N-SHOT

In [ ]:
MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct"
llm = LLM(model=MODEL_NAME, tensor_parallel_size=NUM_GPUS, dtype="bfloat16")

In [17]:
# zero-shot
inputs = [f"###Question:\n{question}\n###Answer:\n" for question in valid_dataset['question']]
outputs = llm.generate(inputs, SamplingParams(temperature=0.0, max_tokens=1024))
short_answers_pred = [extract_last_number_or_ratio(o.outputs[0].text) for o in outputs]
exact_match_score(short_answers_pred, labels)

Processed prompts: 100%|██████████| 500/500 [01:20<00:00,  6.18it/s]


0.228

In [17]:
# zero-shot (instruct)
outputs = llm.generate(chat_inputs, SamplingParams(temperature=0.0, max_tokens=1024))
short_answers_pred = [extract_last_number_or_ratio(o.outputs[0].text) for o in outputs]
exact_match_score(short_answers_pred, labels)

Processed prompts: 100%|██████████| 500/500 [01:07<00:00,  7.41it/s]


0.454

In [ ]:
# 5-shot
few_shot_examples = [f"###Question:\n{ex['question']}\n###Answer:\n{ex['answer']}<stop>" for ex in 
                     dataset.select(range(len(dataset)-5,len(dataset)))]
few_shot_prompt = "\n\n".join(few_shot_examples)
inputs = [few_shot_prompt + "\n\n" + f"###Question:\n{question}\n###Answer:\n" for question in valid_dataset['question']]
outputs = llm.generate(inputs, SamplingParams(temperature=0.0, 
                                              stop_token_ids=[tokenizer.eos_token_id], 
                                              stop=["<stop>"], 
                                              max_tokens=1024))
short_answers_pred = [extract_last_number_or_ratio(o.outputs[0].text) for o in outputs]
exact_match_score(short_answers_pred, labels)

Processed prompts:   9%|▉         | 47/500 [01:06<02:45,  2.73it/s] 

In [30]:
few_shot_examples = [f"###Question:\n{ex['question']}\n###Answer:\n{ex['answer']}<stop>" for ex in 
                     dataset.select(range(len(dataset)-5,len(dataset)))]

In [32]:
few_shot_prompt = "\n\n".join(few_shot_examples)

In [37]:
def fewshot_chat_input(question, answer=None):
    messages = [
        {"role": "system", "content": f"You are an AI assistant that excels in solving math problems. Here are few examples of math problems:\n\n{few_shot_prompt}"},
        {"role": "user", "content": question},
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [38]:
fewshot_chat_inputs = [fewshot_chat_input(question)  for question in valid_dataset['question']]

In [40]:
outputs = llm.generate(fewshot_chat_inputs, SamplingParams(temperature=0.0, 
                                              stop_token_ids=[tokenizer.eos_token_id], 
                                              stop=["<|eot_id|>"],
                                              max_tokens=1024))
short_answers_pred = [extract_last_number_or_ratio(o.outputs[0].text) for o in outputs]
exact_match_score(short_answers_pred, labels)

Processed prompts: 100%|██████████| 500/500 [02:32<00:00,  3.27it/s]


0.452